In [23]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import tools_condition, ToolNode
from langchain_core.messages import HumanMessage
from pprint import pprint

def multiply(a, b):
    """Return the multiplication of two numbers."""
    return a*b

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [multiply]

tool_calls = llm.bind_tools(tools)

class State(MessagesState):
    pass

def tool_calling_model(state: MessagesState):
    return{"messages":[tool_calls.invoke(state["messages"])]}

builder =  StateGraph(State)

builder.add_node("tool_calling_model", tool_calling_model)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START, "tool_calling_model")
builder.add_conditional_edges("tool_calling_model", tools_condition)
builder.add_edge("tools", "tool_calling_model")

memory = MemorySaver()

graph = builder.compile(interrupt_before=["tool_calling_model"], checkpointer=memory)

   

In [ ]:
# User Approval
input_message = {"messages": HumanMessage(content="Multiply 3*3")}
thread = {"configurable":{"thread_id": "2"}}
for event in graph.stream(input_message, thread, stream_mode="values"):
    print(event["messages"][-1])
state  = graph.get_state(thread)
state.next
user_approval = input("Do you want to continue? (yes/no) ")
if user_approval== "yes":
    for event in graph.stream(None, thread, stream_mode="values"):
        event["messages"][-1].pretty_print()
else:
    print("Operation Failed")

content='Multiply 3*3' additional_kwargs={} response_metadata={} id='c9ff38dc-7b29-4416-8bc7-898acd171553'
content='' additional_kwargs={'tool_calls': [{'id': 'call_nHjWO3nsV5M4gxUcRJYp6puN', 'function': {'arguments': '{"a":3,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 61, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-4bbd17c6-db6a-4edd-a240-551c9596c7cd-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 3}, 'id': 'call_nHjWO3nsV5M4gxUcRJYp6puN', 'type': 'tool_call'}] usage_metadata={'input_tokens': 61, 'output_tokens': 18, 'total_tokens': 79, 'input_token_details': {'

In [ ]:
query = {"messages": HumanMessage(content="Multiply 3*3")}
thread = {"configurable":{"thread_id": "2"}}
for event in graph.stream(query, thread, stream_mode="values"):
    print(event["messages"][-1])
# Editing State Fraph before the Assistant for updating the query
graph.update_state(
    thread,
    {"messages": HumanMessage(content="No multiply 3*4")}
)

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1efe94ce-5a6a-6cb6-8001-9c8c716d9cb6'}}

In [26]:
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

No multiply 3*4
================================== Ai Message ==================================
Tool Calls:
  multiply (call_CGPWT48tWn4qBQxoWkC0Fgrr)
 Call ID: call_CGPWT48tWn4qBQxoWkC0Fgrr
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: multiply

12


In [27]:
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================= Tool Message =================================
Name: multiply

12
================================== Ai Message ==================================

The result of multiplying \(3 \times 4\) is \(12\).
